# Домашнее задание к лекции "Базовые понятия статистики"
Будем осуществлять работу с [непростым набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

## Задание 1. Загрузка данных

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names), загрузить его и оставить 8 столбцов для дальнейшего изучения: `surgery?`, `Age`, `rectal temperature`, `pulse`, `respiratory rate`, `temperature of extremities`, `pain`, `outcome`.

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv"
names_with_dtype = [
    ("surgery?", "category"),
    ("age", "int8"),
    ("hospital_number", "int64"),
    ("rectal_temperature", "float64"),
    ("pulse", "float64"),
    ("respiratory_rate", "float64"),
    ("temperature_of_extremities", "category"),
    ("peripheral_pulse", "category"),
    ("mucous_membranes", "category"),
    ("capillary_refill_time", "category"),
    ("pain", "category"),
    ("peristalsis", "category"),
    ("abdominal_distension", "category"),
    ("nasogastric_tube", "category"),
    ("nasogastric_reflux", "category"),
    ("nasogastric_reflux_ph", "float64"),
    ("rectal_examination_feces", "category"),
    ("abdomen", "category"),
    ("packed_cell_volume", "float64"),
    ("total_protein", "float64"),
    ("abdominocentesis_appearance", "category"),
    ("abdomcentesis_total_protein", "float64"),
    ("outcome", "category"),
    ("surgical_lesion?", "category"),
    ("site_of_lesion", "category"),
    ("type_of_lesion", "category"),
    ("subtype_of_lesion", "category"),
    ("code_of_lesion", "category"),
    ("cp_data", "category"),
]
names = [name for name, dtype in names_with_dtype]
dtype = {name: dtype for name, dtype in names_with_dtype}
# prepare na_values
#na_values = {'code_of_lesion': 0, 'subtype_of_lesion': 0}
#[na_values.setdefault(name, '?') for name in names]

horse_data = pd.read_csv(url, names=names, na_values=['?'], dtype=dtype)

# Проверяем, что все признаки с именами

In [2]:
horse_data.head(1).T

,0
surgery?,2
age,1
hospital_number,530101
rectal_temperature,38.5
pulse,66
respiratory_rate,28
temperature_of_extremities,3
peripheral_pulse,3
mucous_membranes,NaN
capillary_refill_time,2


# Оставляем только необходимые признаки

In [3]:
need_attributes = [
    "surgery?",
    "age",
    "rectal_temperature",
    "pulse",
    "respiratory_rate",
    "temperature_of_extremities",
    "pain",
    "outcome",
]
horse_data = horse_data[need_attributes]
horse_data.head()

,surgery?,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2,1,38.5,66.0,28.0,3,5,2
1,1,1,39.2,88.0,20.0,NaN,3,3
2,2,1,38.3,40.0,24.0,1,3,1
3,1,9,39.1,164.0,84.0,4,2,2
4,2,1,37.3,104.0,35.0,NaN,NaN,2


## Задание 2. Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [4]:
discrete_features = ['age']
continuous_features = ['rectal_temperature', 'pulse', 'respiratory_rate']
categorical_features = ['surgery?', 'temperature_of_extremities', 'pain', 'outcome']

# Базовые статистики (количественные типы данных)

In [5]:
horse_data.describe()

,age,rectal_temperature,pulse,respiratory_rate
count,300.000000,240.000000,276.000000,242.000000
mean,1.640000,38.167917,71.913043,30.417355
std,2.173972,0.732289,28.630557,17.642231
min,1.000000,35.400000,30.000000,8.000000
25%,1.000000,37.800000,48.000000,18.500000
50%,1.000000,38.200000,64.000000,24.500000
75%,1.000000,38.500000,88.000000,36.000000
max,9.000000,40.800000,184.000000,96.000000


# Базовые статистики (дискретные и качественные типы данных)

In [6]:
horse_data[discrete_features + categorical_features].mode()

,age,surgery?,temperature_of_extremities,pain,outcome
0,1,1,3,3,1


In [7]:
def outliers(source, feature, action='fiколичественныеколичественныеlter', dropna=False):  
    if dropna:
        df = source.dropna(subset=[feature])
    else:
        df = source
    
    q1 = df[feature].quantile(0.25)
    q3 = df[feature].quantile(0.75)
    iqr = q3 - q1
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    filter_outliers = df[feature].between(lower_bound, upper_bound, inclusive=True)
    
    if action == 'filter':
        return df[filter_outliers]
    elif action == 'show':
        return df[~filter_outliers]

# Выбросы по возрасту
Интерпретация результатов:
* 9 лет - естественное значение, не считаем выбросом

In [8]:
outliers_feature = outliers(horse_data, 'age', 'show')
outliers_feature.age.unique()

array([9], dtype=int8)

# Выбросы по ректальной температуре
* минимальное значение выброса: `35.4` - допустимое значение, говорит о шоковом состоянии(?) (`in late shock`)
* максимальное значение выброса: `40.8` - допустимое значение, говорит о возможной инфекции

In [9]:
outliers_feature = outliers(horse_data, 'rectal_temperature', 'show', True)
outliers_feature.rectal_temperature.agg(['min', 'max'])

min    35.4
max    40.8
Name: rectal_temperature, dtype: float64

# Выбросы по пульсу
* минимальное и максимальное значение превышают нормальный пульс почти в 4-ре раза. 

In [10]:
outliers_feature = outliers(horse_data, 'pulse', 'show', True)
outliers_feature.pulse.agg(['min', 'max'])

min    150.0
max    184.0
Name: pulse, dtype: float64

# Выбросы по частоте дыхания
* минимальное значение выброса превышает норму более чем в 4 раза
* максимальное значение выброса превышает норму более чем в 18 раз

Полезность этих значений сомнительна (?)
>usefulness is doubtful due to the great fluctuations

In [11]:
outliers_feature = outliers(horse_data, 'respiratory_rate', 'show', True)
outliers_feature.pulse.agg(['min', 'max'])

min     42.0
max    184.0
Name: pulse, dtype: float64

# Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

# Считаем пропуски

In [12]:
horse_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   surgery?                    299 non-null    category
 1   age                         300 non-null    int8    
 2   rectal_temperature          240 non-null    float64 
 3   pulse                       276 non-null    float64 
 4   respiratory_rate            242 non-null    float64 
 5   temperature_of_extremities  244 non-null    category
 6   pain                        245 non-null    category
 7   outcome                     299 non-null    category
dtypes: category(4), float64(3), int8(1)
memory usage: 9.2 KB


In [13]:
(horse_data.isna().mean() * 100).round(2).sort_values()

age                            0.00
surgery?                       0.33
outcome                        0.33
pulse                          8.00
pain                          18.33
temperature_of_extremities    18.67
respiratory_rate              19.33
rectal_temperature            20.00
dtype: float64

## Считаем пропуски по группам

In [14]:
# удаляем пропуски по surgery и outcome
horse_data.dropna(subset=['surgery?', 'outcome'], inplace=True)
(horse_data.isna().mean() * 100).round(2).sort_values()

surgery?                       0.00
age                            0.00
outcome                        0.00
pulse                          8.03
pain                          18.39
temperature_of_extremities    18.73
respiratory_rate              19.40
rectal_temperature            20.07
dtype: float64

## Сгруппируем по (age)

In [15]:
features = ['age']
# количество
horse_data.groupby(features).count()

,surgery?,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
age,,,,,,,
1,275,219,253,221,227,227,275
9,24,20,22,20,16,17,24


In [16]:
# процент
(horse_data.set_index(features).isna().groupby(features).mean() * 100).round(2)

,surgery?,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
age,,,,,,,
1,0,20.36,8.00,19.64,17.45,17.45,0
9,0,16.67,8.33,16.67,33.33,29.17,0


### Заполняем пропуски
Для группы `age=9` имеется всего 24 записи, дальнейшее выделение подгрупп не имеет смысла, т.к. они получатся очень маленькие.

Заполним пропуски медианой в этой группе. 

In [17]:
selected_group = horse_data.age == 9
selected_data = horse_data[selected_group]
# медиана
filled_by_median = selected_data.fillna(selected_data.groupby(features).transform('median'))
horse_data.fillna(filled_by_median, inplace=True)

# убедимся, что получилось
(horse_data.set_index(features).isna().groupby(features).mean() * 100).round(2)

,surgery?,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
age,,,,,,,
1,0,20.36,8.0,19.64,17.45,17.45,0
9,0,0.00,0.0,0.00,33.33,29.17,0


### Мода. Две штуки. Что делать? 
Пока отложим....

In [18]:
selected_data[discrete_features+categorical_features].set_index(['age', 'surgery?', 'outcome']).mode()

,temperature_of_extremities,pain
0,1,2
1,3,3


## Сгруппируем по (age, surgery?)

In [19]:
features = ['age', 'surgery?']
# количество
horse_data.groupby(features).count()

rectal_temperature  pulse  respiratory_rate  \
age surgery?                                                
1   1                        125    148               129   
    2                         94    105                92   
9   1                         18     18                18   
    2                          6      6                 6   

              temperature_of_extremities  pain  outcome  
age surgery?                                             
1   1                                127   129      162  
    2                                100    98      113  
9   1                                 13    14       18  
    2                                  3     3        6

In [20]:
# процент
(horse_data.set_index(features).isna().groupby(features).mean() * 100).round(2)

rectal_temperature  pulse  respiratory_rate  \
age surgery?                                                
1   1                      22.84   8.64             20.37   
    2                      16.81   7.08             18.58   
9   1                       0.00   0.00              0.00   
    2                       0.00   0.00              0.00   

              temperature_of_extremities   pain  outcome  
age surgery?                                              
1   1                              21.60  20.37        0  
    2                              11.50  13.27        0  
9   1                              27.78  22.22        0  
    2                              50.00  50.00        0

## Сгруппируем по (age, surgery?, outcome)

In [21]:
features = ['age', 'surgery?', 'outcome']
# количество
horse_data.groupby(features).count()

rectal_temperature  pulse  respiratory_rate  \
age surgery? outcome                                                
1   1        1                      74.0   82.0              70.0   
             2                      31.0   40.0              36.0   
             3                      20.0   26.0              23.0   
    2        1                      68.0   73.0              67.0   
             2                      13.0   16.0              12.0   
             3                      13.0   16.0              13.0   
9   1        1                       7.0    7.0               7.0   
             2                      10.0   10.0              10.0   
             3                       1.0    1.0               1.0   
    2        1                       5.0    5.0               5.0   
             2                       1.0    1.0               1.0   
             3                       NaN    NaN               NaN   

                      temperature_of_extremities  pain  
age surgery? outcome                                    
1   1        1                              71.0  70.0  
             2                              39.0  40.0  
             3                              17.0  19.0  
    2        1                              69.0  67.0  
             2                              16.0  16.0  
             3                              15.0  15.0  
9   1        1                               4.0   5.0  
             2                               8.0   8.0  
             3                               1.0   1.0  
    2        1                               2.0   2.0  
             2                               1.0   1.0  
             3                               NaN   NaN

In [22]:
# процент
(horse_data.set_index(features).isna().groupby(features).mean() * 100).round(2)

rectal_temperature  pulse  respiratory_rate  \
age surgery? outcome                                                
1   1        1                     15.91   6.82             20.45   
             2                     35.42  16.67             25.00   
             3                     23.08   0.00             11.54   
    2        1                     12.82   6.41             14.10   
             2                     27.78  11.11             33.33   
             3                     23.53   5.88             23.53   
9   1        1                      0.00   0.00              0.00   
             2                      0.00   0.00              0.00   
             3                      0.00   0.00              0.00   
    2        1                      0.00   0.00              0.00   
             2                      0.00   0.00              0.00   
             3                       NaN    NaN               NaN   

                      temperature_of_extremities   pain  
age surgery? outcome                                     
1   1        1                             19.32  20.45  
             2                             18.75  16.67  
             3                             34.62  26.92  
    2        1                             11.54  14.10  
             2                             11.11  11.11  
             3                             11.76  11.76  
9   1        1                             42.86  28.57  
             2                             20.00  20.00  
             3                              0.00   0.00  
    2        1                             60.00  60.00  
             2                              0.00   0.00  
             3                               NaN    NaN

### Заполняем пропуски
Заполним пропуски медианой в этих группах

In [23]:
# медиана
filled_by_median = horse_data.fillna(horse_data.groupby(features).transform('median'))
horse_data.fillna(filled_by_median, inplace=True)

# убедимся, что получилось
(horse_data.set_index(features).isna().groupby(features).mean() * 100).round(2)

rectal_temperature pulse respiratory_rate  \
age surgery? outcome                                             
1   1        1                        0     0                0   
             2                        0     0                0   
             3                        0     0                0   
    2        1                        0     0                0   
             2                        0     0                0   
             3                        0     0                0   
9   1        1                        0     0                0   
             2                        0     0                0   
             3                        0     0                0   
    2        1                        0     0                0   
             2                        0     0                0   
             3                      NaN   NaN              NaN   

                      temperature_of_extremities   pain  
age surgery? outcome                                     
1   1        1                             19.32  20.45  
             2                             18.75  16.67  
             3                             34.62  26.92  
    2        1                             11.54  14.10  
             2                             11.11  11.11  
             3                             11.76  11.76  
9   1        1                             42.86  28.57  
             2                             20.00  20.00  
             3                              0.00   0.00  
    2        1                             60.00  60.00  
             2                              0.00   0.00  
             3                               NaN    NaN

# Заполним остатки пропусков
Для `temperature_of_extremities` и `pain` возьмем моду по всему набору данных.
###### устал я :(
Иначе, можно было бы пройтись по каждой весомой группе и выделить в них свою моду...

In [24]:
features = ['age', 'surgery?', 'outcome']
# количество
horse_data.groupby(features).count()

rectal_temperature  pulse  respiratory_rate  \
age surgery? outcome                                                
1   1        1                      88.0   88.0              88.0   
             2                      48.0   48.0              48.0   
             3                      26.0   26.0              26.0   
    2        1                      78.0   78.0              78.0   
             2                      18.0   18.0              18.0   
             3                      17.0   17.0              17.0   
9   1        1                       7.0    7.0               7.0   
             2                      10.0   10.0              10.0   
             3                       1.0    1.0               1.0   
    2        1                       5.0    5.0               5.0   
             2                       1.0    1.0               1.0   
             3                       NaN    NaN               NaN   

                      temperature_of_extremities  pain  
age surgery? outcome                                    
1   1        1                              71.0  70.0  
             2                              39.0  40.0  
             3                              17.0  19.0  
    2        1                              69.0  67.0  
             2                              16.0  16.0  
             3                              15.0  15.0  
9   1        1                               4.0   5.0  
             2                               8.0   8.0  
             3                               1.0   1.0  
    2        1                               2.0   2.0  
             2                               1.0   1.0  
             3                               NaN   NaN

# Заполним остатки пропусков
Для `temperature_of_extremities` и `pain` возьмем моду 

In [25]:
horse_data.pain.fillna(horse_data.pain.mode()[0], inplace=True)
horse_data.temperature_of_extremities.fillna(horse_data.temperature_of_extremities.mode()[0], inplace=True)

horse_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   surgery?                    299 non-null    category
 1   age                         299 non-null    int8    
 2   rectal_temperature          299 non-null    float64 
 3   pulse                       299 non-null    float64 
 4   respiratory_rate            299 non-null    float64 
 5   temperature_of_extremities  299 non-null    category
 6   pain                        299 non-null    category
 7   outcome                     299 non-null    category
dtypes: category(4), float64(3), int8(1)
memory usage: 21.4 KB
